In [1]:
import numpy as np
import copy
import keras.models

from MCTS import MCTS
from Game import Game 
from model import AmoebaZeroModel
import params


Using TensorFlow backend.


In [2]:
HISTORY_SIZE     = params.PIPELINE_HISTORY_SIZE
BATCH_SIZE       = params.PIPELINE_BATCH_SIZE
TRAINING_LOOP_NR = params.PIPELINE_TRAINING_LOOP_NR
SELFPLAY_NR      = params.PIPELINE_SELFPLAY_NR
EVAL_NR          = params.PIPELINE_EVAL_NR
THRESHOLD        = params.PIPELINE_THRESHOLD

class Pipeline:
    def __init__(self, verbose = 0):
        self.verbose = verbose
        
        self.history_input = np.zeros( (HISTORY_SIZE,15,15,4))
        self.history_p     = np.zeros( (HISTORY_SIZE,225))
        self.history_v     = np.zeros( (HISTORY_SIZE))
        self.batch_input   = np.zeros( (BATCH_SIZE,15,15,4))
        self.batch_p       = np.zeros( (BATCH_SIZE,225))
        self.batch_v       = np.zeros( (BATCH_SIZE))
        self.next_index    = 0
        self.history_nr    = 0
        self.version       = 0
        
        self.model_trained = AmoebaZeroModel()
        self.model_trained.model.save("./models/saved_model_0.h5")
        
        self.model_best       = AmoebaZeroModel()
        self.model_best.model = keras.models.load_model("./models/saved_model_0.h5")
    
    def main_loop(self):
        nr = 0
        self.self_play()
        while True:
            print("Loop - ", nr)
            self.train()
            if self.evaluate_trained():
                self.version += 1
                self.model_trained.model.save("./models/saved_model_"+str(self.version)+".h5")
                self.model_best.model = keras.models.load_model("./models/saved_model_"+str(self.version)+".h5")
                print("New model saved. Version: ", self.version)

                self.self_play()
                
    def self_play(self):
        if self.verbose >= 1:
            print("Self play")
        for i in range(SELFPLAY_NR):
            if self.verbose >=2:
                print(i, "start_index = ", self.next_index)
            start_index = self.next_index
            g = Game()
            mcts = MCTS(g, self.model_best.evaluate)
            nr = 0
            while (g.winner==0):
                nr += 1
                x,y,n = mcts.select_move()                
                g.fill_grids_for_nn(self.history_input,self.next_index)
                mcts.fill_p(self.history_p,self.next_index)
                self.next_index = (self.next_index + 1) % HISTORY_SIZE
                if self.history_nr<HISTORY_SIZE:
                    self.history_nr += 1
                g.move(x,y)
                mcts.reinit(g,n)
                if self.verbose>=3:
                    print(nr, "- move: ", x, y, "; next_index: ", self.next_index)
                if i==0 and self.verbose>=4:
                    g.print()
            v = g.get_final_value()

            if self.verbose >= 2:
                print("Result:", g.winner)
            
            i = start_index
            while i != self.next_index:
                self.history_v[i] = v
                i = (i+1)%HISTORY_SIZE
    
    def train(self):
        if self.verbose >= 1:
            print("Train")
        for i in range(TRAINING_LOOP_NR):
            if self.verbose >= 2:
                print("i: " + str(i))
            
            for j in range(BATCH_SIZE):
                ind = np.random.randint(0,self.history_nr)
                self.batch_input[j] = self.history_input[ind]
                self.batch_p[j]     = self.history_p[ind]
                self.batch_v[j]     = self.history_v[ind]
            self.model_trained.train(self.batch_input,[self.batch_p,self.batch_v],BATCH_SIZE)           
            
    def evaluate_trained(self):
        if self.verbose >= 1:
            print("Evaluate")

        balance = 0 #+ if trained, - if best won more
        
        #best begins
        for i in range(EVAL_NR//2):
            if self.verbose >= 2:
                print("i: ",i)
            g  = Game()
            nr = 0
            while (g.winner==0):
                if nr%2==0:
                    mcts = MCTS(g, self.model_best.evaluate)
                    x,y,_ = mcts.select_move(competitive = True)
                    g.move(x,y)
                else:
                    mcts = MCTS(g, self.model_trained.evaluate)
                    x,y,_ = mcts.select_move(competitive = True)
                    g.move(x,y)
                nr+=1
            if g.winner == 1:
                balance -= 1
            elif g.winner == 2:
                balance += 1

            if self.verbose >= 2:
                print("Result:", balance)

        #trained begins
        for i in range(EVAL_NR//2):
            if self.verbose >= 2:
                print("i: ",i)
            g  = Game()
            nr = 0
            while (g.winner==0):
                if nr%2==0:
                    mcts = MCTS(g, self.model_trained.evaluate)
                    x,y,_ = mcts.select_move(competitive = True)
                    g.move(x,y)
                else:
                    mcts = MCTS(g, self.model_best.evaluate)
                    x,y,_ = mcts.select_move(competitive = True)
                    g.move(x,y)
                nr+=1
            if g.winner == 1:
                balance += 1
            elif g.winner == 2:
                balance -= 1

            if self.verbose >= 2:
                print("Result:", g.winner)

                
        if self.verbose >= 1:
            print("Balance: ",balance," Thr.: ",THRESHOLD)
                
        return balance >= THRESHOLD
           
        
        

In [3]:
pl = Pipeline(verbose = 2)
pl.main_loop()


Self play
0 start_index =  0
Result: 2
1 start_index =  154
Result: 2
2 start_index =  280
Result: 2
Loop -  0
Train
i: 0
Epoch 1/1
64/64 [==============================] - 1s 8ms/step - loss: 8.5953 - out_p_loss: 5.6466 - out_v_loss: 0.1282 - out_p_categorical_crossentropy: 5.6466 - out_v_mean_squared_error: 0.1282
i: 1
Epoch 1/1
64/64 [==============================] - 0s 458us/step - loss: 8.5029 - out_p_loss: 5.6366 - out_v_loss: 0.0854 - out_p_categorical_crossentropy: 5.6366 - out_v_mean_squared_error: 0.0854
i: 2
Epoch 1/1
64/64 [==============================] - 0s 520us/step - loss: 8.3366 - out_p_loss: 5.5523 - out_v_loss: 0.0424 - out_p_categorical_crossentropy: 5.5523 - out_v_mean_squared_error: 0.0424
i: 3
Epoch 1/1
64/64 [==============================] - 0s 477us/step - loss: 8.3100 - out_p_loss: 5.5803 - out_v_loss: 0.0262 - out_p_categorical_crossentropy: 5.5803 - out_v_mean_squared_error: 0.0262
i: 4
Epoch 1/1
64/64 [==============================] - 0s 420us/step - l

i: 39
Epoch 1/1
64/64 [==============================] - 0s 416us/step - loss: 6.7293 - out_p_loss: 5.1150 - out_v_loss: 1.6491e-04 - out_p_categorical_crossentropy: 5.1150 - out_v_mean_squared_error: 1.6491e-04
i: 40
Epoch 1/1
64/64 [==============================] - 0s 390us/step - loss: 6.6826 - out_p_loss: 5.0907 - out_v_loss: 1.5531e-04 - out_p_categorical_crossentropy: 5.0907 - out_v_mean_squared_error: 1.5531e-04
i: 41
Epoch 1/1
64/64 [==============================] - 0s 463us/step - loss: 6.7012 - out_p_loss: 5.1313 - out_v_loss: 1.8752e-04 - out_p_categorical_crossentropy: 5.1313 - out_v_mean_squared_error: 1.8752e-04
i: 42
Epoch 1/1
64/64 [==============================] - 0s 428us/step - loss: 6.6325 - out_p_loss: 5.0842 - out_v_loss: 1.6748e-04 - out_p_categorical_crossentropy: 5.0842 - out_v_mean_squared_error: 1.6748e-04
i: 43
Epoch 1/1
64/64 [==============================] - 0s 405us/step - loss: 6.5998 - out_p_loss: 5.0727 - out_v_loss: 2.0305e-04 - out_p_categorical_

i: 78
Epoch 1/1
64/64 [==============================] - 0s 464us/step - loss: 6.0108 - out_p_loss: 5.0123 - out_v_loss: 9.5912e-04 - out_p_categorical_crossentropy: 5.0123 - out_v_mean_squared_error: 9.5912e-04
i: 79
Epoch 1/1
64/64 [==============================] - 0s 471us/step - loss: 5.9069 - out_p_loss: 4.9190 - out_v_loss: 6.2296e-04 - out_p_categorical_crossentropy: 4.9190 - out_v_mean_squared_error: 6.2296e-04
i: 80
Epoch 1/1
64/64 [==============================] - 0s 397us/step - loss: 5.8631 - out_p_loss: 4.8852 - out_v_loss: 6.3883e-04 - out_p_categorical_crossentropy: 4.8852 - out_v_mean_squared_error: 6.3883e-04
i: 81
Epoch 1/1
64/64 [==============================] - 0s 414us/step - loss: 5.9164 - out_p_loss: 4.9484 - out_v_loss: 6.9916e-04 - out_p_categorical_crossentropy: 4.9484 - out_v_mean_squared_error: 6.9916e-04
i: 82
Epoch 1/1
64/64 [==============================] - 0s 466us/step - loss: 5.8061 - out_p_loss: 4.8478 - out_v_loss: 7.3370e-04 - out_p_categorical_

i: 14
Epoch 1/1
64/64 [==============================] - 0s 466us/step - loss: 5.4147 - out_p_loss: 4.6779 - out_v_loss: 6.9311e-04 - out_p_categorical_crossentropy: 4.6779 - out_v_mean_squared_error: 6.9311e-04
i: 15
Epoch 1/1
64/64 [==============================] - 0s 403us/step - loss: 5.3969 - out_p_loss: 4.6649 - out_v_loss: 6.5647e-04 - out_p_categorical_crossentropy: 4.6649 - out_v_mean_squared_error: 6.5647e-04
i: 16
Epoch 1/1
64/64 [==============================] - 0s 527us/step - loss: 5.3769 - out_p_loss: 4.6496 - out_v_loss: 6.4723e-04 - out_p_categorical_crossentropy: 4.6496 - out_v_mean_squared_error: 6.4723e-04
i: 17
Epoch 1/1
64/64 [==============================] - 0s 421us/step - loss: 5.3581 - out_p_loss: 4.6351 - out_v_loss: 8.0772e-04 - out_p_categorical_crossentropy: 4.6351 - out_v_mean_squared_error: 8.0772e-04
i: 18
Epoch 1/1
64/64 [==============================] - 0s 452us/step - loss: 5.3969 - out_p_loss: 4.6784 - out_v_loss: 6.2492e-04 - out_p_categorical_

i: 53
Epoch 1/1
64/64 [==============================] - 0s 551us/step - loss: 5.2586 - out_p_loss: 4.6425 - out_v_loss: 7.0719e-04 - out_p_categorical_crossentropy: 4.6425 - out_v_mean_squared_error: 7.0719e-04
i: 54
Epoch 1/1
64/64 [==============================] - 0s 495us/step - loss: 5.1169 - out_p_loss: 4.5029 - out_v_loss: 5.9329e-04 - out_p_categorical_crossentropy: 4.5029 - out_v_mean_squared_error: 5.9329e-04
i: 55
Epoch 1/1
64/64 [==============================] - 0s 593us/step - loss: 5.1779 - out_p_loss: 4.5657 - out_v_loss: 5.9695e-04 - out_p_categorical_crossentropy: 4.5657 - out_v_mean_squared_error: 5.9695e-04
i: 56
Epoch 1/1
64/64 [==============================] - 0s 522us/step - loss: 5.1110 - out_p_loss: 4.5005 - out_v_loss: 7.5271e-04 - out_p_categorical_crossentropy: 4.5005 - out_v_mean_squared_error: 7.5271e-04
i: 57
Epoch 1/1
64/64 [==============================] - 0s 584us/step - loss: 5.1529 - out_p_loss: 4.5443 - out_v_loss: 6.6022e-04 - out_p_categorical_

i: 92
Epoch 1/1
64/64 [==============================] - 0s 480us/step - loss: 4.9230 - out_p_loss: 4.3533 - out_v_loss: 6.5072e-04 - out_p_categorical_crossentropy: 4.3533 - out_v_mean_squared_error: 6.5072e-04
i: 93
Epoch 1/1
64/64 [==============================] - 0s 445us/step - loss: 4.9906 - out_p_loss: 4.4212 - out_v_loss: 7.9440e-04 - out_p_categorical_crossentropy: 4.4212 - out_v_mean_squared_error: 7.9440e-04
i: 94
Epoch 1/1
64/64 [==============================] - 0s 461us/step - loss: 4.8665 - out_p_loss: 4.2976 - out_v_loss: 6.2688e-04 - out_p_categorical_crossentropy: 4.2976 - out_v_mean_squared_error: 6.2688e-04
i: 95
Epoch 1/1
64/64 [==============================] - 0s 429us/step - loss: 4.9744 - out_p_loss: 4.4057 - out_v_loss: 6.7408e-04 - out_p_categorical_crossentropy: 4.4057 - out_v_mean_squared_error: 6.7408e-04
i: 96
Epoch 1/1
64/64 [==============================] - 0s 448us/step - loss: 4.9014 - out_p_loss: 4.3330 - out_v_loss: 7.0506e-04 - out_p_categorical_

KeyboardInterrupt: 